# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import pickle
import numpy as np
import re
#import xgboost as xgb

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, classification_report, precision_recall_fscore_support, confusion_matrix
#from sklearn import XGBModel

pd.set_option('max_rows', 1000)
pd.set_option('max_columns', 1000)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load data from database
#def load_data():
engine = create_engine('sqlite:///processed_etl_pipeline_data.db')
df = pd.read_sql_table('processed_etl_pipeline_data', engine)

#Drop any null records
df = df.dropna()

X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
#return df, X, Y

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    """
    Purpose:  Normalize and clean message text for modeling by removing stems and endings, standardizing text and formatting
    
    Input: string (message data)

    Output: List of cleaned strings
    """

    # Normalize by converting to lowercase and removing punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize
    words = word_tokenize(text)

    # Remove stopwords
    words = [w for w in words if w not in stopwords.words('english')]

    # Stem word tokens
    stemmer = PorterStemmer()
    stemmed = [stemmer.stem(word) for word in words]
    
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
                    ('vector', CountVectorizer(tokenizer = tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(RandomForestClassifier()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size = 0.7, test_size = 0.3, random_state = 42)

np.random.seed(42)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vector', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        str...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def get_performance_metrics(y_test, y_preds, col_names):
    """
    Purpose: Generate performance metrics for model
    
    Inputs:
    y_test:  actual data values
    y_preds:  model predicted data values
    col_names: list containing all Y category column names for each predicted category
    
    Returns: dataframe showing actual and predicted above, plus accuracy, precision, recall, and F1
    """
    # Create blank list to hold all outcomes from loop
    all_metrics = []
    
    # Loop to generate stats for each column
    for i in range(len(col_names)):
        accuracy = accuracy_score(y_test[:,i], y_preds[:,i], normalize = True)
        precision = precision_score(y_test[:,i], y_preds[:,i], average = 'micro')
        recall = recall_score(y_test[:,i], y_preds[:,i], average = 'micro')
        f1 = f1_score(y_test[:,i], y_preds[:,i], average = 'micro')
        
        # Append all metrics to blank list
        all_metrics.append([accuracy, precision, recall, f1])
    
    # Convert list to dataframe
    performance_df = pd.DataFrame(np.array(all_metrics), index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1 Score'])
    
    return performance_df

In [9]:
#Evaluate performance for training set
y_train_preds = pipeline.predict(X_train)

#Create list of column names for function
col_names = list(Y.columns)

print(get_performance_metrics(np.array(y_train), y_train_preds, col_names))

                        Accuracy  Precision    Recall  F1 Score
related                 0.977206   0.977206  0.977206  0.977206
request                 0.962291   0.962291  0.962291  0.962291
offer                   0.999719   0.999719  0.999719  0.999719
aid_related             0.966371   0.966371  0.966371  0.966371
medical_help            0.984944   0.984944  0.984944  0.984944
medical_products        0.990573   0.990573  0.990573  0.990573
search_and_rescue       0.994512   0.994512  0.994512  0.994512
security                0.995497   0.995497  0.995497  0.995497
military                0.998171   0.998171  0.998171  0.998171
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.980864   0.980864  0.980864  0.980864
food                    0.969185   0.969185  0.969185  0.969185
shelter                 0.971859   0.971859  0.971859  0.971859
clothing                0.997608   0.997608  0.997608  0.997608
money                   0.995779   0.995

In [10]:
# Repeat evaluation for test set
y_test_preds = pipeline.predict(X_test)

print(get_performance_metrics(np.array(y_test), y_test_preds, col_names))

                        Accuracy  Precision    Recall  F1 Score
related                 0.611293   0.611293  0.611293  0.611293
request                 0.615890   0.615890  0.615890  0.615890
offer                   0.999015   0.999015  0.999015  0.999015
aid_related             0.600788   0.600788  0.600788  0.600788
medical_help            0.941563   0.941563  0.941563  0.941563
medical_products        0.964544   0.964544  0.964544  0.964544
search_and_rescue       0.979317   0.979317  0.979317  0.979317
security                0.986868   0.986868  0.986868  0.986868
military                0.996389   0.996389  0.996389  0.996389
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.918910   0.918910  0.918910  0.918910
food                    0.843401   0.843401  0.843401  0.843401
shelter                 0.885424   0.885424  0.885424  0.885424
clothing                0.991464   0.991464  0.991464  0.991464
money                   0.986540   0.986

In [11]:
# See how true target variable observations are split among Y categories predicted
Y.sum()/len(Y)

related                   0.680784
request                   0.350832
offer                     0.000985
aid_related               0.384615
medical_help              0.055747
medical_products          0.033488
search_and_rescue         0.020684
security                  0.012804
military                  0.004334
child_alone               0.000000
water                     0.077613
food                      0.148232
shelter                   0.104403
clothing                  0.009061
money                     0.012312
missing_people            0.008372
refugees                  0.016153
death                     0.024328
other_aid                 0.141928
infrastructure_related    0.030040
transport                 0.018812
buildings                 0.036147
electricity               0.006501
tools                     0.002955
hospitals                 0.005319
shops                     0.002659
aid_centers               0.007485
other_infrastructure      0.016547
weather_related     

We see responses are highly imbalanced between categories predicted.  This makes it more difficult to predict categories with smaller positive class proportions.  It also means any positive predictions in our test set should be closely checked against the proportionality of actual members, as the performance could just be mirroring the proportionality rather than actually predicting correctly.

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {'clf__estimator__max_depth': [5, 15]}#,
            # 'clf__estimator__n_estimators': [5, 15],
            # 'clf__estimator__max_features': [10, 15]}

cv = GridSearchCV(pipeline, param_grid = parameters)

np.random.seed(42)
grid_search_model = cv.fit(X_train, y_train)

In [13]:
# See all grid search results
grid_search_model.cv_results_

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

{'mean_fit_time': array([ 16.65757871,  17.06446576]),
 'std_fit_time': array([ 0.03911839,  0.04935915]),
 'mean_score_time': array([ 8.02841457,  8.03691093]),
 'std_score_time': array([ 0.0606768 ,  0.03099756]),
 'param_clf__estimator__max_depth': masked_array(data = [5 15],
              mask = [False False],
        fill_value = ?),
 'params': [{'clf__estimator__max_depth': 5},
  {'clf__estimator__max_depth': 15}],
 'split0_test_score': array([ 0.18108907,  0.17729   ]),
 'split1_test_score': array([ 0.17982271,  0.17686788]),
 'split2_test_score': array([ 0.18657661,  0.18488814]),
 'mean_test_score': array([ 0.18249613,  0.179682  ]),
 'std_test_score': array([ 0.00293129,  0.00368532]),
 'rank_test_score': array([1, 2], dtype=int32),
 'split0_train_score': array([ 0.18341072,  0.18742085]),
 'split1_train_score': array([ 0.18446602,  0.19164204]),
 'split2_train_score': array([ 0.18087801,  0.18573238]),
 'mean_train_score': array([ 0.18291825,  0.18826509]),
 'std_train_score

In [14]:
# Look for best performing model
grid_search_model.best_params_

{'clf__estimator__max_depth': 5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
y_test_preds_tuned = grid_search_model.predict(X_test)

print(get_performance_metrics(np.array(y_test), y_test_preds_tuned, col_names))

                        Accuracy  Precision    Recall  F1 Score
related                 0.661195   0.661195  0.661195  0.661195
request                 0.648720   0.648720  0.648720  0.648720
offer                   0.999015   0.999015  0.999015  0.999015
aid_related             0.613920   0.613920  0.613920  0.613920
medical_help            0.943533   0.943533  0.943533  0.943533
medical_products        0.967170   0.967170  0.967170  0.967170
search_and_rescue       0.979317   0.979317  0.979317  0.979317
security                0.986868   0.986868  0.986868  0.986868
military                0.996389   0.996389  0.996389  0.996389
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.926789   0.926789  0.926789  0.926789
food                    0.853907   0.853907  0.853907  0.853907
shelter                 0.894944   0.894944  0.894944  0.894944
clothing                0.991464   0.991464  0.991464  0.991464
money                   0.987196   0.987

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Try XG Boost
xg_pipeline = Pipeline([
                        ('vector', CountVectorizer(tokenizer = tokenize)),
                        ('tfidf', TfidfTransfomer()),
                        ('xgb', MultiOutputClassifier(xgb.XGBClassifier()))
                        ])

xg_params = {'max_depth': [5, 10],
             'n_estimators': [5, 10]}

xg_cv = GridSearchCV(xg_pipeline, xg_params)

In [ ]:
# Optimize model
np.random.seed(42)
xg_model = xg_cv.fit(X_train, Y_train)

In [ ]:
# Evaluate grid search model results
xg_model.cv_results_

In [ ]:
# Find parameters that generated best performance
xg_model.best_params_

In [ ]:
# Predict on test set
y_test_preds_xg = xg_model.predict(X_test)

print(get_performance_metrics(np.array(y_test), y_test_preds, col_names))

### 9. Export your model as a pickle file

In [16]:
pickle.dump(grid_search_model, open('disaster_rf_clf_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.